In [ ]:
from IPython.display import display, Markdown, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
import math
import ROOT
import numpy as np, pandas as pd
import yaml
import base64
import ctypes
import logging

import bokeh
from bokeh.io import output_notebook, show
from bokeh.plotting import output_file

from RootInteractive.InteractiveDrawing.bokeh.bokehDrawSA import *
from RootInteractive.InteractiveDrawing.bokeh.bokehTools import bokehDrawArray
from RootInteractive.Tools.pandaTools import initMetadata
from RootInteractive.Tools.aliTreePlayer import *
from RootInteractive.Tools.compressArray import arrayCompressionRelative8

output_notebook()

# Display skimmed AO2D tracks

In [ ]:
#inputFile="/lustre/alice/users/miranov/NOTESData/alice-tpc-notes/JIRA/ATO-592/tpcqcskimmingTracks.root"
inputFile="~/Software/TPCQCVis/data/tpcqcskimmingTracks.root"

In [ ]:
f=ROOT.TFile.Open(inputFile)
tracks = f.Get("tracks;2")  
tracks.SetAlias("weight1","((triggerMask&0x1)>0)*max(weight,0.01)")
tracks.SetAlias("rMin","trackPar.mX")
tracks.SetAlias("qPt","trackPar.mP[4]")
tracks.SetAlias("pzPt","trackPar.mP[3]")
tracks.SetAlias("itsOn","itsClusterMap>0")
tracks.SetAlias("tofOn","abs(tofChi2)<100")
#tracks.SetAlias("side","pzPt>0")
#tracks.Show(0)

In [ ]:
df=tree2Panda(tracks, [".*"], "", exclude=["trackPar"])
df["side"]=df.eval("pzPt>0") ##  (A/C/CrossAC/CrossCA side)
df["isPrim5"]=df.eval("abs(dcaXY)<5 &  abs(dcaZ)<5 & rMin<5")
df["tpcNCR"]=df["tpcNClsFindable"]-df["tpcNClsFindableMinusCrossedRows"]
df.head()

In [ ]:
#df["NCR"].min()

In [ ]:
widgetParams=[['range', ['pt']],
              ['range', ['qPt']],
              ['range', ['phi']],
              ['range', ['pzPt']],
              #
              ['range', ['dcaXY']],
              ['range', ['dcaZ']],
              ['range', ['rMin']],
              ['range', ['tpcNClsFindable']],
              #
              ['multiSelect',["side"]],
              ['multiSelect',["isPrim5"]],
              ['multiSelect',["itsOn"]],
              ['multiSelect',["tofOn"]],
              ['multiSelect',["hasCollision"]],
             ]
widgetLayoutDesc=[[0,1,2,3],[4,5,6,7],[8,9,10,11,12], {'sizing_mode': 'scale_width'}]   
#widgetLayoutDesc=[[0,1,2,3],[4,5,6], {'sizing_mode': 'scale_width'}]  
#widgetLayoutDesc=[[0],[1], {'sizing_mode': 'scale_width'}] 

In [ ]:
aliasArray = [    {
        "name": "logPtWeight",
        "variables": ["bin_count"],
        "func": "return Math.log(bin_count)",
        "context": "ptWeightedHist"
    }
]
parameterArray = [
    {"name": "size", "value":7, "range":[0, 30]},
    {"name": "legendFontSize", "value":"13px", "options":["9px", "11px", "13px", "15px"]},
]

In [ ]:
histArrayPhi=[
    {"name": "dcaXYVsPhi", "variables": ["phi","dcaXY"], "nbins":[30,80], "range": [None, [-6,6]], "quantiles": [.05, .5, .95],"axis": [0, 1]},
    {"name": "dcaZVsPhi", "variables": ["phi","dcaZ"], "nbins":[30,80], "range": [None, [-6,6]], "quantiles": [.05, .5, .95],"axis": [0, 1]},
    {"name": "tpcNClsFindableVsPhi", "variables": ["phi","tpcNClsFindable"], "nbins":[30,20], "range": [None,[0, 159]], "quantiles": [.05, .5, .95],"axis": [0, 1]},
    {"name": "tpcNCRVsPhi", "variables": ["phi","tpcNCR"], "nbins":[30,20], "range": [None,[0, 159]], "quantiles": [.05, .5, .95],"axis": [0, 1]},
    {"name": "tpcChi2NClVsPhi", "variables": ["phi","tpcChi2NCl"], "nbins":[30,20], "range": [None,[0, 6.28]], "quantiles": [.05, .5, .95],"axis": [0, 1]},
]

histoArray = [
    {"name": "ptHist", "variables": ["pt"], "nbins":200, "range": [0, 20]},
              {"name": "ptWeightedHist", "variables": ["pt"], "nbins":200, "range": [0, 20], "weights":"weight"},
              {"name": "weightHist", "variables": ["weight"], "nbins":200, "range": [0, 20]},
              {"name": "ptWeightHist", "variables": ["pt","weight"], "nbins":[200,200], "range": [[0, 20],[0,20]]},
              {"name": "phiWeightedHist", "variables": ["phi"], "nbins":180, "range": [0, 6.28], "weights":"weight"},
              {"name": "phiHist", "variables": ["phi"], "nbins":180, "range": [0, 6.28]},
              {"name": "phiHistA", "variables": ["phi"], "nbins":90},
              {"name": "ptVsPhi", "variables": ["pt","phi"], "nbins":[20,18], "range": [[0,20],[0, 6.28]]}
]
histoArray.extend(histArrayPhi)

In [ ]:
histoArrayLog=[]
for his in histoArray:
    histoName=his["name"]    
    histoArrayLog.append({ "name": f"log{histoName}","variables": ["bin_count"],"func": "return Math.log(bin_count+0.2)","context": f"{histoName}"})
    histoArrayLog.append({ "name": f"log{histoName}Err","variables": ["bin_count"],"func": "return 1/Math.sqrt(bin_count+0.2)","context": f"{histoName}"})               
    histoArrayLog.append({ "name": f"{histoName}Err","variables": ["entries","std"],"func": "return std/Math.sqrt(entries+0.1)","context": f"{histoName}_1"})  
aliasArray.extend(histoArrayLog)

In [ ]:
df.shape

In [ ]:
output_file("investigating_skimmed_tracks.html")
from RootInteractive.Tools.compressArray import arrayCompressionRelative8

figureArray = [
                # [['bin_center_0'], [''], {"size":"size","source":"ptWeightedHist"}],
    #[['phi'],['dcaXYVsPhi']],
    #[['phi'],['dcaZVsPhi']],
    #[['phi'],['tpcNClsFindableVsPhi']],
    #[['phi'],['tpcNCRVsPhi']],
    #[['phi'],['tpcChi2NClVsPhi']],
    #
    [["bin_center_1"],["logdcaXYVsPhi"],{"source": "dcaXYVsPhi","colorZvar":"bin_center_0","errY":"logdcaXYVsPhiErr"}],
    [["bin_center_1"],["logdcaZVsPhi"],{"source": "dcaZVsPhi","colorZvar":"bin_center_0","errY":"logdcaZVsPhiErr"}],
    [["bin_center_0"],["quantile_1"],{"source": "dcaXYVsPhi_1","errY":"dcaXYVsPhiErr"}],
    [["bin_center_0"],["quantile_1"],{"source": "dcaZVsPhi_1","errY":"dcaZVsPhiErr"}],
    #
    [["bin_center_1"],["logtpcNClsFindableVsPhi"],{"source": "tpcNClsFindableVsPhi","colorZvar":"bin_center_0","errY":"logtpcNClsFindableVsPhiErr"}],
    [["bin_center_1"],["logtpcNCRVsPhi"],{"source": "tpcNCRVsPhi","colorZvar":"bin_center_0","errY":"logtpcNCRVsPhiErr"}],
    [["bin_center_1"],["logtpcChi2NClVsPhi"],{"source": "tpcChi2NClVsPhi","colorZvar":"bin_center_0","errY":"logtpcChi2NClVsPhiErr"}],
    [["bin_center_0"],["quantile_1"],{"source": "tpcNClsFindableVsPhi_1"}],
    [["bin_center_0"],["quantile_1"],{"source": "tpcNCRVsPhi_1"}],
    [["bin_center_0"],["quantile_1"],{"source": "tpcChi2NClVsPhi_1"}],
    {'plot_height':150, "commonX":0}
]          

figureLayout = {
    "DCA phi":[[0,1],[2,3], {'plot_height':200,'sizing_mode':'scale_width',"legend_visible":True}],
    "TPC N/Chi2, phi":[[4,5,6],[7,8,9], {'plot_height':300,'sizing_mode':'scale_width',"legend_visible":True}]
}
tooltips = []

  
xxx = bokehDrawSA.fromArray(df, "pt<20&abs(qPt)<20&abs(pzPt)<2", figureArray, widgetParams, layout=figureLayout, tooltips=tooltips, widgetLayout=widgetLayoutDesc,
                            histogramArray=histoArray,aliasArray=aliasArray,sizing_mode='scale_width',arrayCompression=arrayCompressionRelative8)

In [ ]:
output_file("investigating_skimmed_tracksBackup.html")

figureArray = [[['pt'],['ptHist']],
               [['pt'],['ptWeightedHist']],
               [['bin_center'],['logPtWeight'],{"source":"ptWeightedHist"}],
               [['pt'],['ptVsPhi']],
               [['phi'],['phiWeightedHist','phiHist']],
               [['phi'],['phiHistA']],
              ]

layout = [[0,2],[3,5]]
tooltips = []

  
xxx = bokehDrawSA.fromArray(df, "(triggerMask%2)>0&pt<20&abs(qPt)<20", figureArray, widgetParams, layout=layout, tooltips=tooltips, widgetLayout=widgetLayoutDesc,
                            histogramArray=histoArray,aliasArray=aliasArray)

In [ ]:
c1=ROOT.TCanvas("c","c")
tracks.SetAlias("itsOn","(flags&0x1)>0")
tracks.Draw("tpcNClsFindable","")
c1.Draw()

In [ ]:
tracks.GetLeaf("tpcNClsFindable").Print()
tracks.GetLeaf("tpcNClsFindableMinusFound").Print()

In [ ]:
c1=ROOT.TCanvas("c","c")
tracks.SetAlias("itsOn","(flags&0x1)>0")
tracks.Draw("dcaXY:phi","pzPt<0&&phi<1&&abs(dcaXY)<10&&pt>2","prof")
c1.Draw()

## New version

In [ ]:
inputFile="~/Software/TPCQCVis/data/tpcqcskimmingTracks.root"
f=ROOT.TFile.Open(inputFile)
tracks = f.Get("tracks;2")  
tracks.SetAlias("weight1","((triggerMask&0x1)>0)*max(weight,0.01)")
tracks.SetAlias("rMin","trackPar.mX")
tracks.SetAlias("qPt","trackPar.mP[4]")
tracks.SetAlias("pzPt","trackPar.mP[3]")
tracks.SetAlias("itsOn","itsClusterMap>0")
tracks.SetAlias("tofOn","abs(tofChi2)<100")

In [ ]:
df=tree2Panda(tracks, [".*"], "", exclude=["trackPar"])
df["side"]=df.eval("pzPt>0") ##  (A/C/CrossAC/CrossCA side)
df["isPrim5"]=df.eval("abs(dcaXY)<5 &  abs(dcaZ)<5 & rMin<5")
df["tpcNCR"]=df["tpcNClsFindable"]-df["tpcNClsFindableMinusCrossedRows"]
df.head()

In [ ]:
aliasArray = [
    {
        "name": "logPtWeight",
        "variables": ["bin_count"],
        "func": "return Math.log(bin_count)",
        "context": "ptWeightedHist"
    }
]
parameterArray = [
    {"name": "size", "value":5, "range":[0, 20]},
    {"name": "legendFontSize", "value":"13px", "options":["9px", "11px", "13px", "15px"]},
]

In [ ]:
widgetParams=[
      ['range', ['pt']],
      ['range', ['qPt']],
      ['range', ['phi']],
      ['range', ['pzPt']],
      #
      ['range', ['dcaXY']],
      ['range', ['dcaZ']],
      ['range', ['rMin']],
      ['range', ['tpcNClsFindable']],
      #
      ['multiSelect',["side"]],
      ['multiSelect',["isPrim5"]],
      ['multiSelect',["itsOn"]],
      ['multiSelect',["tofOn"]],
      ['multiSelect',["hasCollision"]],
      # Graphics
      ['slider',["size"], {"callback": "parameter"}],
      ['select',["legendFontSize"], {"callback": "parameter", "default": 2}],
             ]
widgetLayoutDesc={
    "Selection:": [[0,1,2,3],[4,5,6,7],[8,9,10,11,12], {'sizing_mode': 'scale_width'}],
    "Graphics": [[13, 14], {'sizing_mode': 'scale_width'}],
}

In [ ]:
histoArray = []

histArrayPhi=[
    {"name": "dcaXYVsPhi", "variables": ["phi","dcaXY"], "nbins":[30,80], "range": [None, [-6,6]], "quantiles": [.05, .5, .95],"axis": [0, 1]},
    {"name": "dcaZVsPhi", "variables": ["phi","dcaZ"], "nbins":[30,80], "range": [None, [-6,6]], "quantiles": [.05, .5, .95],"axis": [0, 1]},
    {"name": "tpcNClsFindableVsPhi", "variables": ["phi","tpcNClsFindable"], "nbins":[30,20], "range": [None,[0, 159]], "quantiles": [.05, .5, .95],"axis": [0, 1]},
    {"name": "tpcNCRVsPhi", "variables": ["phi","tpcNCR"], "nbins":[30,20], "range": [None,[0, 159]], "quantiles": [.05, .5, .95],"axis": [0, 1]},
    {"name": "tpcChi2NClVsPhi", "variables": ["phi","tpcChi2NCl"], "nbins":[30,20], "range": [None,[0, 6.28]], "quantiles": [.05, .5, .95],"axis": [0, 1]},
]
histoArray.extend(histArrayPhi)

histArrayPt=[
    {"name": "dcaXYVsPt", "variables": ["pt","dcaXY"], "nbins":[30,80], "range": [None, [-6,6]], "quantiles": [.05, .5, .95],"axis": [0, 1]},
    {"name": "dcaZVsPt", "variables": ["pt","dcaZ"], "nbins":[30,80], "range": [None, [-6,6]], "quantiles": [.05, .5, .95],"axis": [0, 1]},
    {"name": "tpcNClsFindableVsPt", "variables": ["pt","tpcNClsFindable"], "nbins":[30,20], "range": [None,[0, 159]], "quantiles": [.05, .5, .95],"axis": [0, 1]},
    {"name": "tpcNCRVsPt", "variables": ["pt","tpcNCR"], "nbins":[30,20], "range": [None,[0, 159]], "quantiles": [.05, .5, .95],"axis": [0, 1]},
    {"name": "tpcChi2NClVsPt", "variables": ["pt","tpcChi2NCl"], "nbins":[30,20], "range": [None,[0, 6.28]], "quantiles": [.05, .5, .95],"axis": [0, 1]},
]
histoArray.extend(histArrayPt)

histArraytpcNCR=[
    {"name": "dcaXYVstpcNCR", "variables": ["tpcNCR","dcaXY"], "nbins":[30,80], "range": [None, [-6,6]], "quantiles": [.05, .5, .95],"axis": [0, 1]},
    {"name": "dcaZVstpcNCR", "variables": ["tpcNCR","dcaZ"], "nbins":[30,80], "range": [None, [-6,6]], "quantiles": [.05, .5, .95],"axis": [0, 1]},
    #{"name": "tpcNClsFindableVsPt", "variables": ["pt","tpcNClsFindable"], "nbins":[30,20], "range": [None,[0, 159]], "quantiles": [.05, .5, .95],"axis": [0, 1]},
    #{"name": "tpcNCRVsPt", "variables": ["pt","tpcNCR"], "nbins":[30,20], "range": [None,[0, 159]], "quantiles": [.05, .5, .95],"axis": [0, 1]},
    #{"name": "tpcChi2NClVsPt", "variables": ["pt","tpcChi2NCl"], "nbins":[30,20], "range": [None,[0, 6.28]], "quantiles": [.05, .5, .95],"axis": [0, 1]},
]
histoArray.extend(histArraytpcNCR)

histoArrayLog=[]
for his in histoArray:
    histoName=his["name"]    
    histoArrayLog.append({ "name": f"log{histoName}","variables": ["bin_count"],"func": "return Math.log(bin_count+0.2)","context": f"{histoName}"})
    histoArrayLog.append({ "name": f"log{histoName}Err","variables": ["bin_count"],"func": "return 1/Math.sqrt(bin_count+0.2)","context": f"{histoName}"})               
    histoArrayLog.append({ "name": f"{histoName}Err","variables": ["entries","std"],"func": "return std/Math.sqrt(entries+0.1)","context": f"{histoName}_1"})
aliasArray.extend(histoArrayLog)

In [ ]:
output_file("investigating_skimmed_tracks.html")

figureArray = [
    # Phi DCA
    [["bin_center_1"],["logdcaXYVsPhi"],{"source": "dcaXYVsPhi","colorZvar":"bin_center_0","errY":"logdcaXYVsPhiErr","xAxisTitle":"DCA XY","yAxisTitle":"log(counts)"}],
    [["bin_center_1"],["logdcaZVsPhi"],{"source": "dcaZVsPhi","colorZvar":"bin_center_0","errY":"logdcaZVsPhiErr","xAxisTitle":"DCA Z","yAxisTitle":"log(counts)"}],
    [["bin_center_0"],["quantile_1"],{"source": "dcaXYVsPhi_1","errY":"dcaXYVsPhiErr","xAxisTitle":"Phi"}],
    [["bin_center_0"],["quantile_1"],{"source": "dcaZVsPhi_1","errY":"dcaZVsPhiErr","xAxisTitle":"Phi"}],
    # Phi
    [["bin_center_1"],["logtpcNClsFindableVsPhi"],{"source": "tpcNClsFindableVsPhi","colorZvar":"bin_center_0","errY":"logtpcNClsFindableVsPhiErr","xAxisTitle":"NClsFindable"}],
    [["bin_center_1"],["logtpcNCRVsPhi"],{"source": "tpcNCRVsPhi","colorZvar":"bin_center_0","errY":"logtpcNCRVsPhiErr","xAxisTitle":"TPC NCR"}],
    [["bin_center_1"],["logtpcChi2NClVsPhi"],{"source": "tpcChi2NClVsPhi","colorZvar":"bin_center_0","errY":"logtpcChi2NClVsPhiErr"}],
    [["bin_center_0"],["quantile_1"],{"source": "tpcNClsFindableVsPhi_1"}],
    [["bin_center_0"],["quantile_1"],{"source": "tpcNCRVsPhi_1"}],
    [["bin_center_0"],["quantile_1"],{"source": "tpcChi2NClVsPhi_1"}],
    # Pt DCA
    [["bin_center_1"],["logdcaXYVsPt"],{"source": "dcaXYVsPt","colorZvar":"bin_center_0","errY":"logdcaXYVsPtErr","xAxisTitle":"DCA XY","yAxisTitle":"log(counts)"}],
    [["bin_center_1"],["logdcaZVsPt"],{"source": "dcaZVsPt","colorZvar":"bin_center_0","errY":"logdcaZVsPtErr","xAxisTitle":"DCA Z","yAxisTitle":"log(counts)"}],
    [["bin_center_0"],["quantile_1"],{"source": "dcaXYVsPt_1","errY":"dcaXYVsPtErr","xAxisTitle":"pT"}],
    [["bin_center_0"],["quantile_1"],{"source": "dcaZVsPt_1","errY":"dcaZVsPtErr","xAxisTitle":"pT"}],
    # Pt
    [["bin_center_1"],["logtpcNClsFindableVsPt"],{"source": "tpcNClsFindableVsPt","colorZvar":"bin_center_0","errY":"logtpcNClsFindableVsPtErr","xAxisTitle":"NClsFindable"}],
    [["bin_center_1"],["logtpcNCRVsPt"],{"source": "tpcNCRVsPt","colorZvar":"bin_center_0","errY":"logtpcNCRVsPtErr","xAxisTitle":"TPC NCR"}],
    [["bin_center_1"],["logtpcChi2NClVsPt"],{"source": "tpcChi2NClVsPt","colorZvar":"bin_center_0","errY":"logtpcChi2NClVsPtErr"}],
    [["bin_center_0"],["quantile_1"],{"source": "tpcNClsFindableVsPt_1"}],
    [["bin_center_0"],["quantile_1"],{"source": "tpcNCRVsPt_1"}],
    [["bin_center_0"],["quantile_1"],{"source": "tpcChi2NClVsPt_1"}],
    # tpcNCR DCA
    [["bin_center_1"],["logdcaXYVstpcNCR"],{"source": "dcaXYVstpcNCR","colorZvar":"bin_center_0","errY":"logdcaXYVstpcNCRErr","xAxisTitle":"DCA XY","yAxisTitle":"log(counts)"}],
    [["bin_center_1"],["logdcaZVstpcNCR"],{"source": "dcaZVstpcNCR","colorZvar":"bin_center_0","errY":"logdcaZVstpcNCRErr","xAxisTitle":"DCA Z","yAxisTitle":"log(counts)"}],
    [["bin_center_0"],["quantile_1"],{"source": "dcaXYVstpcNCR_1","errY":"dcaXYVstpcNCRErr","xAxisTitle":"NCR"}],
    [["bin_center_0"],["quantile_1"],{"source": "dcaZVstpcNCR_1","errY":"dcaZVstpcNCRErr","xAxisTitle":"NCR"}],
    {'plot_height':150, "commonX":0, "size":"size", "legend_options": {"label_text_font_size": "legendFontSize"}}
]          

figureLayout = {
    "Phi": {
        "DCA phi":[[0,1],[2,3], {'plot_height':200,'sizing_mode':'scale_width',"legend_visible":True}],
        "TPC N/Chi2, phi":[[4,5,6],[7,8,9], {'plot_height':300,'sizing_mode':'scale_width',"legend_visible":True}]
    },
    "Pt": {
        "DCA pt":[[10,11],[12,13], {'plot_height':200,'sizing_mode':'scale_width',"legend_visible":True}],
        "TPC N/Chi2, pt":[[14,15,16],[17,18,19], {'plot_height':300,'sizing_mode':'scale_width',"legend_visible":True}]
    },
    "tpcNCR": {
        "DCA pt":[[20,21],[22,23], {'plot_height':200,'sizing_mode':'scale_width',"legend_visible":True}]
    }
}
tooltips = []  

In [ ]:
xxx = bokehDrawSA.fromArray(df, "pt<20&abs(qPt)<20&abs(pzPt)<2", figureArray, widgetParams, layout=figureLayout, tooltips=tooltips, widgetLayout=widgetLayoutDesc,
                            parameterArray=parameterArray, histogramArray=histoArray,aliasArray=aliasArray,sizing_mode='scale_width',
                            arrayCompression=arrayCompressionRelative8)